# Titanic Survival Prediction Analysis

## Table of Contents

### 1. Project Overview
- Problem statement and business questions
- Dataset description
- Project objectives

### 2. Setup and Data Loading  
- Library imports
- Loading Titanic dataset
- Initial data inspection

### 3. Exploratory Data Analysis (EDA)
- Data structure and types
- Basic statistics
- Missing values analysis
- Target variable distribution

### 4. Data Cleaning: Handling Missing Values
- Age imputation strategy
- Embarked missing value handling
- Deck column decision
- Verification of cleaning

### 5. Feature Engineering
- Creating family_size feature
- Creating is_alone indicator
- Creating age_group categories
- Creating fare_per_person
- Creating social_category

### 6. Data Visualization
- Survival by gender
- Survival by passenger class  
- Age distribution by survival
- Fare vs Age analysis
- Key insights from visualizations

### 7. Data Preparation for Machine Learning [NEXT]
- Feature selection
- Encoding categorical variables
- Train-test split
- Feature scaling

### 8. Model Building
- Logistic Regression implementation
- Random Forest Classifier
- Model training and validation

### 9. Model Evaluation
- Performance metrics comparison
- Feature importance analysis
- Best model selection

### 10. Conclusions & Recommendations
- Key findings summary
- Business insights
- Project limitations
- Future work

---


# Titanic Survival Prediction - Complete Analysis

##  <a id="project-overview"></a>
# 1.Project Overview

This project analyzes the Titanic dataset to predict passenger survival using machine learning. We'll explore the data, handle missing values, create new features, and build predictive models.

**Business Questions:**
1. What factors most influenced survival on the Titanic?
2. Can we accurately predict survival using machine learning?
3. Which demographic groups had the highest/lowest survival rates?

**Dataset:** Titanic passenger data from Kaggle (891 passengers, 12 features)
**Target Variable:** `survived` (0 = did not survive, 1 = survived)

## 2. Setup and Data Loading
<a id="setup--data-loading"></a>

The first step is to import all necessary libraries for data analysis, visualization, and machine learning.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

### 1.1 Load the Dataset

The next step is to load the Titanic dataset which is being used for this analysis, from a CSV file. The dataset is being uploaded from Github repository automatically.

In [ ]:


your_github_url = "https://raw.githubusercontent.com/tanatswanjanji18-afk/Titanic-Predictive-Analysis/refs/heads/main/Titanic.csv"
df = pd.read_csv(your_github_url)

print("Loaded from GitHub repository")
df.head()



## 3. Exploratory Data Analysis (EDA)

Before building any models, we need to understand our data. Let's examine:
- Data structure and types
- Missing values
- Basic statistics
- Target variable distribution

In [ ]:


print(" Data Exploration")
print("=" * 50)

# Basic info
print("1. Data Types:")
print(df.info())

print("\n2. Dataset Statistics:")
print(df.describe())

print("\n3. Missing Values check:")
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing': missing,
    'Percent': missing_percent
})
print(missing_df[missing_df['Missing'] > 0])

print("\n4. Target Variable - Survival:")
survival_rate = df['survived'].mean() * 100
print(f"   Overall survival rate: {survival_rate:.1f}%")
print(f"   Did not survive: {(df['survived'] == 0).sum()} passengers")
print(f"   Survived: {(df['survived'] == 1).sum()} passengers")

## 4. Data Cleaning: Handling Missing Values

We identified several columns with missing values:

| Column | Missing Count | Percentage | Action |
|--------|--------------|------------|---------|
| age | 177 | 19.9% | Estimate with median by passenger class |
| embarked | 2 | 0.2% | Fill with most common port |
| deck | 688 | 77.2% | Drop column (too many missing) |
| embark_town | 2 | 0.2% | Fill with most common town |

**Why these choices?**
- **Age**: 20% missing is significant, but we can make educated guesses based on passenger class
- **Embarked**: Only 2 missing, safe to use the most common value
- **Deck**: 77% missing is too high for reliable estimation, so we remove it

In [ ]:
print("=" * 50)
print("Cleaning Data")
print("=" * 50)


df_clean = df.copy()

print("BEFORE cleaning:")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])
print()


embarked_mode = df_clean['embarked'].mode()[0]
df_clean['embarked'] = df_clean['embarked'].fillna(embarked_mode)
print(f"'embarked': Filled 2 missing with '{embarked_mode}'")


embark_town_mode = df_clean['embark_town'].mode()[0]
df_clean['embark_town'] = df_clean['embark_town'].fillna(embark_town_mode)
print(f"'embark_town': Filled 2 missing with '{embark_town_mode}'")


df_clean = df_clean.drop('deck', axis=1)
print("'deck': Dropped column (77% missing - too unreliable)")


print("\n Analyzing age")
age_by_class = df_clean.groupby('pclass')['age'].median()
print(f"Median age by class:\n{age_by_class}")


def fill_age(row):
    if pd.isnull(row['age']):
        return age_by_class[row['pclass']]
    return row['age']

df_clean['age'] = df_clean.apply(fill_age, axis=1)
print(f"'age': Filled 177 missing with class-based medians")

print("\n cleaning:")
missing_after = df_clean.isnull().sum()
if missing_after.sum() == 0:
    print("Cleaned Data")
else:
    print(f"Still missing: {missing_after[missing_after > 0]}")

## 5. Feature Engineering

Creating new features can improve model performance. Below created are :

1. **family_size**: Total family members onboard (siblings + spouses + parents + children + self)
   - Why? Family size likely affected survival chances

2. **is_alone**: Binary indicator if passenger was traveling alone
   - Why? Solo travelers might have had different survival rates

3. **age_group**: Categorical age groups (child, teen, adult, senior)
   - Why? Age affects survival differently across life stages

4. **fare_per_person**: Fare divided by family size
   - Why? Accounts for group tickets vs individual tickets

5. **social_category**: Extracted from 'who' (Mr, Mrs, Miss, Master, Rare)
   - Why? Social title indicates age, gender, and social status

In [ ]:
# ============================================
# FEATURE ENGINEERING
# ============================================

print("Creating new features for improved prediction...")
print("-" * 50)


df_clean['family_size'] = df_clean['sibsp'] + df_clean['parch'] + 1
print(f"Total family members (sibsp + parch + 1)")
print(f" {df_clean['family_size'].min()} to {df_clean['family_size'].max()} members")


print("\n2. 'is_alone'...")
df_clean['is_alone'] = (df_clean['family_size'] == 1).astype(int)
print(f" Added: Binary indicator (1 if traveling alone)")
print(f" {df_clean['is_alone'].sum()} passengers ({df_clean['is_alone'].mean()*100:.1f}%) were alone")


print("\n3.'age_group'...")

bins = [0, 12, 18, 35, 60, 100]
labels = ['child', 'teen', 'young_adult', 'adult', 'senior']

df_clean['age_group'] = pd.cut(df_clean['age'], bins=bins, labels=labels)
print(f"Categorical age groups")
print(f" Distribution:")
for group in labels:
    count = (df_clean['age_group'] == group).sum()
    percent = count / len(df_clean) * 100
    print(f"     {group:12}: {count:3} passengers ({percent:.1f}%)")


print("\n4.'fare_per_person'...")
df_clean['fare_per_person'] = df_clean['fare'] / df_clean['family_size']
print(f" Added: Fare divided by family size")
print(f"Average: ${df_clean['fare_per_person'].mean():.2f} per person")


print("\n5.'social_category'...")

if 'who' in df_clean.columns:

    df_clean['social_category'] = df_clean['who']
    print(f" Using 'who' column as social category")
    print(f" Categories: {df_clean['social_category'].unique().tolist()}")
else:

    print("  NA")
    df_clean['social_category'] = df_clean.apply(
        lambda row: 'child' if row['age'] < 12 else ('woman' if row['sex'] == 'female' else 'man'),
        axis=1
    )
    print(f"   Created social categories from sex and age")


print(f"{df_clean.shape}")
print(f"New features added ({len([col for col in df_clean.columns if col not in df.columns])} total):")


original_cols = set(df.columns)
new_cols = [col for col in df_clean.columns if col not in original_cols]

for i, col in enumerate(new_cols, 1):
    print(f"{i:2}. {col:20} → {df_clean[col].dtype}")

print("\n Sample of data with new features:")
sample_cols = ['survived', 'sex', 'age', 'family_size', 'is_alone',
               'age_group', 'fare_per_person', 'social_category', ]
display(df_clean[sample_cols].head(8))

print("\nSurvival rates by new features:")
print("1. By family size:")
for size in sorted(df_clean['family_size'].unique())[:6]:  # Show first 6
    group = df_clean[df_clean['family_size'] == size]
    if len(group) > 0:
        rate = group['survived'].mean() * 100
        print(f"   Size {size}: {rate:.1f}% survival ({len(group)} passengers)")

print("\n2. By age group:")
for group in df_clean['age_group'].cat.categories:
    data = df_clean[df_clean['age_group'] == group]
    rate = data['survived'].mean() * 100
    print(f"   {group:12}: {rate:.1f}% survival")

print("\n3. Traveling alone vs with family:")
alone_rate = df_clean[df_clean['is_alone'] == 1]['survived'].mean() * 100
family_rate = df_clean[df_clean['is_alone'] == 0]['survived'].mean() * 100
print(f"   Alone: {alone_rate:.1f}% survival")
print(f"   With family: {family_rate:.1f}% survival")

## 6. Data Visualization

Visualizations help us understand patterns and relationships in the data. We'll create four key plots:

1. **Survival by Gender**: Compare male vs female survival rates
2. **Survival by Class**: How passenger class affected survival
3. **Age Distribution**: Age patterns for survivors vs non-survivors
4. **Fare vs Age**: Relationship between fare, age, and survival

In [ ]:
print("=" * 50)
print("Data Visualization")
print("=" * 50)


fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Survival by Gender
sns.countplot(x='sex', hue='survived', data=df_clean, ax=axes[0,0])
axes[0,0].set_title('Survival by Gender')
axes[0,0].set_xlabel('Gender')
axes[0,0].set_ylabel('Count')

# Plot 2: Survival by Passenger Class
sns.countplot(x='pclass', hue='survived', data=df_clean, ax=axes[0,1])
axes[0,1].set_title('Survival by Passenger Class')
axes[0,1].set_xlabel('Class (1=First, 2=Second, 3=Third)')
axes[0,1].set_ylabel('Count')

# Plot 3: Age Distribution by Survival
sns.histplot(data=df_clean, x='age', hue='survived',
             kde=True, bins=30, ax=axes[1,0])
axes[1,0].set_title('Age Distribution by Survival')
axes[1,0].set_xlabel('Age')
axes[1,0].set_ylabel('Density')

# Plot 4: Fare vs Age colored by Survival
scatter = axes[1,1].scatter(df_clean['age'], df_clean['fare'],
                           c=df_clean['survived'], alpha=0.6, cmap='coolwarm')
axes[1,1].set_title('Fare vs Age (Color = Survived)')
axes[1,1].set_xlabel('Age')
axes[1,1].set_ylabel('Fare')
plt.colorbar(scatter, ax=axes[1,1], label='Survived (0=No, 1=Yes)')

plt.tight_layout()
plt.show()

print("\nKey Insights :")
print("1. Women had much higher survival rates than men")
print("2. First-class passengers had better survival chances")
print("3. Children (especially under 12) had higher survival rates")
print("4. Higher fare passengers (likely first class) survived more")

<a id="data-preparation-for-machine-learning"></a>
## 7. Data Preparation for Machine Learning

Before building predictive models, we need to prepare our cleaned data:

### **Reasons for Cleaning Data:**
1. **Machine learning algorithms require numerical input** - We must convert text categories to numbers
2. **Features should be on similar scales** - Large differences in ranges can cause models to be biased
3. **We need separate data for training and testing** - To evaluate model performance fairly
4. **Not all features are equally useful** - Some of the features from the dataset may be redundant or irrelevant

### **Preparation:**
1. **Feature Selection** - Choose which columns to use for prediction
2. **Categorical Encoding** - Convert text (sex, embarked, etc.) to numerical values
3. **Train-Test Split** - Separate data for model training (80%) and testing (20%)
4. **Feature Scaling** - Normalize numerical features to similar ranges

### **Expected Outcome:**
Clean, formatted data ready for machine learning algorithms with proper training/testing separation.

In [ ]:
# ============================================
# 7. DATA PREPARATION FOR MACHINE LEARNING
# ============================================

print("Preparing Data for Machine Learning")
print("=" * 50)


df_model = df_clean.copy()

print("Available features in cleaned dataset:")
print("-" * 40)
for i, col in enumerate(df_model.columns, 1):
    print(f"{i:2}. {col:20} ({df_model[col].dtype})")

# ============================================
# STEP 1: Select Features for Modeling
# ============================================

print("\n1️SELECTING FEATURES FOR PREDICTION")
print("-" * 40)


exclude_features = [
    'survived',
    'alive',
    'embark_town',
    'class',
    'who',
    'alone',
    'deck' ]


available_features = [col for col in df_model.columns if col not in exclude_features]

print(f"Selected {len(available_features)} features for modeling:")
for i, feature in enumerate(available_features, 1):
    print(f"   {i:2}. {feature}")


X = df_model[available_features]
y = df_model['survived']

print(f"\nFeature matrix (X) shape: {X.shape}")
print(f"Target vector (y) shape: {y.shape}")



print("\n2️Encoding Categorical Variables")
print("-" * 40)


categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

if categorical_cols:
    print(f"Found {len(categorical_cols)} categorical columns to encode:")
    for col in categorical_cols:
        unique_vals = X[col].unique()[:5]
        print(f"   • {col}: {len(X[col].unique())} unique values → {list(unique_vals)}...")


    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
    print(f"\nOne-hot encoding complete")
    print(f"   Before: {X.shape[1]} features")
    print(f"   After:  {X_encoded.shape[1]} features")
else:
    print("No categorical columns found - skipping encoding")
    X_encoded = X.copy()

# ============================================
# STEP 3: Split Data into Train and Test Sets
# ============================================

print("\n3️SPLITTING DATA INTO TRAINING & TESTING SETS")
print("-" * 40)

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


print(f"   • Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"   • Testing set:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.0f}%)")
print(f"\n Survival rate in each set:")
print(f"   • Training: {y_train.mean()*100:.1f}% survived ({y_train.sum()} survivors)")
print(f"   • Testing:  {y_test.mean()*100:.1f}% survived ({y_test.sum()} survivors)")

# ============================================
# STEP 4: Scale Numerical Features
# ============================================

print("\n4️Scaling Numerical Features")
print("-" * 40)

from sklearn.preprocessing import StandardScaler


numerical_cols = X_encoded.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Found {len(numerical_cols)} numerical columns to scale:")
for col in numerical_cols[:10]:  # Show first 10
    print(f"   • {col}: mean={X_train[col].mean():.2f}, std={X_train[col].std():.2f}")


scaler = StandardScaler()


X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()


X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])

print(f"\n Feature scaling complete using StandardScaler")
print(f"   • Training data now has mean≈0, std≈1 for all numerical features")
print(f"   • Same transformation applied to testing data")

# ============================================
# FINAL SUMMARY
# ============================================


print(f"\n Final Data Shapes:")
print(f"   X_train: {X_train_scaled.shape}")
print(f"   X_test:  {X_test_scaled.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   y_test:  {y_test.shape}")

print(f"\n Feature Names ({X_train_scaled.shape[1]} total):")
feature_names = X_train_scaled.columns.tolist()
for i in range(0, len(feature_names), 5):  # Show 5 per line
    print(f"   {', '.join(feature_names[i:i+5])}")

print(f"\n Sample Training Data(first 3 rows):")
display(X_train_scaled.head(3))



Preparing Data for Machine Learning
Available features in cleaned dataset:
----------------------------------------
 1. Unnamed: 0           (int64)
 2. survived             (int64)
 3. pclass               (int64)
 4. sex                  (object)
 5. age                  (float64)
 6. sibsp                (int64)
 7. parch                (int64)
 8. fare                 (float64)
 9. embarked             (object)
10. class                (object)
11. who                  (object)
12. adult_male           (bool)
13. embark_town          (object)
14. alive                (object)
15. alone                (bool)
16. family_size          (int64)
17. is_alone             (int64)
18. age_group            (category)
19. fare_per_person      (float64)
20. social_category      (object)

1️SELECTING FEATURES FOR PREDICTION
----------------------------------------
Selected 14 features for modeling:
    1. Unnamed: 0
    2. pclass
    3. sex
    4. age
    5. sibsp
    6. parch
    7. fare
    8

,Unnamed: 0,pclass,age,sibsp,parch,fare,adult_male,family_size,is_alone,fare_per_person,sex_male,embarked_Q,embarked_S,age_group_teen,age_group_young_adult,age_group_adult,age_group_senior,social_category_man,social_category_woman
692,0.966222,0.829568,-0.394592,-0.465084,-0.466183,0.513812,True,-0.556339,0.800346,0.957826,True,False,True,False,True,False,False,True,False
481,0.146119,-0.370945,-0.017217,-0.465084,-0.466183,-0.662563,True,-0.556339,0.800346,-0.541189,True,False,True,False,True,False,False,True,False
527,0.324909,-1.571457,0.586583,-0.465084,-0.466183,3.955399,True,-0.556339,0.800346,5.343327,True,False,True,False,False,True,False,True,False


<a id="model-building"></a>
## 8. Model Building & Implementation

For this predictive analysis project, I'll implement and compare three fundamental machine learning algorithms to predict Titanic survival:

### **Models Selected:**
1. **Logistic Regression** - Simple baseline classifier, easy to interpret
2. **Decision Tree** - Basic tree-based model, good for understanding feature importance
3. **Random Forest** - Ensemble of decision trees, more robust but still understandable

### **Reasons for Models Selected ?**
- **Logistic Regression**: Provides a clear baseline and interpretable coefficients
- **Decision Tree**: Visualizable, shows clear decision rules
- **Random Forest**: Improves upon decision trees while remaining interpretable

In [ ]:
# ============================================
# 8. MODEL BUILDING & IMPLEMENTATION
# ============================================

print("Building Machine Learning Models")
print("=" * 50)


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

print("Imported machine learning libraries")
print("   • Logistic Regression - Basic classification")
print("   • Decision Tree - Simple tree-based model")
print("   • Random Forest - Ensemble of trees")


# Model 1: Logistic Regression
# ============================================

print("\n" + "="*50)
print("1. Logistic Regression")
print("="*50)

print("Training Logistic Regression model...")
start_time = time.time()


logreg = LogisticRegression(
    random_state=42,
    max_iter=1000,
    solver='lbfgs'
)
logreg.fit(X_train_scaled, y_train)

training_time = time.time() - start_time
print(f"✓ Training completed in {training_time:.2f} seconds")


y_pred_logreg = logreg.predict(X_test_scaled)


accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print(f"✓ Test Accuracy: {accuracy_logreg:.3f} ({accuracy_logreg*100:.1f}%)")


print("\nClassification Report:")
print("-" * 60)
print(classification_report(y_test, y_pred_logreg,
                           target_names=['Did Not Survive', 'Survived']))


# Model 2: Decision Tree
# ============================================

print("\n" + "="*50)
print("2. Decision Tree Classifier")
print("="*50)

print("Training Decision Tree model...")
start_time = time.time()


tree = DecisionTreeClassifier(
    random_state=42,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5
)
tree.fit(X_train_scaled, y_train)

training_time = time.time() - start_time
print(f"✓ Training completed in {training_time:.2f} seconds")


y_pred_tree = tree.predict(X_test_scaled)


accuracy_tree = accuracy_score(y_test, y_pred_tree)
print(f"✓ Test Accuracy: {accuracy_tree:.3f} ({accuracy_tree*100:.1f}%)")


print("\n Decision Tree Structure :")
print(f"   • Tree depth: {tree.get_depth()}")
print(f"   • Number of leaves: {tree.get_n_leaves()}")
print(f"   • Number of features used: {sum(tree.feature_importances_ > 0)}")


# Model 3: Random Forest
# ============================================

print("\n" + "="*50)
print("3. Random Forest Classifier ")
print("="*50)


start_time = time.time()


forest = RandomForestClassifier(
    n_estimators=50,
    max_depth=8,
    random_state=42,
    min_samples_split=10,
    min_samples_leaf=5,
    n_jobs=-1
)
forest.fit(X_train_scaled, y_train)

training_time = time.time() - start_time
print(f"✓ Training completed in {training_time:.2f} seconds")
print(f"   (Trained {forest.n_estimators} decision trees)")


y_pred_forest = forest.predict(X_test_scaled)


accuracy_forest = accuracy_score(y_test, y_pred_forest)
print(f"✓ Test Accuracy: {accuracy_forest:.3f} ({accuracy_forest*100:.1f}%)")


# Model Comparison
# ============================================

print("\n" + "="*50)
print(" Comparison Summary")
print("="*50)


comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree', 'Random Forest'],
    'Accuracy': [accuracy_logreg, accuracy_tree, accuracy_forest],
    'Interpretability': ['High', 'High', 'Medium'],
    'Complexity': ['Low', 'Medium', 'High']
})


comparison['Accuracy (%)'] = comparison['Accuracy'].apply(lambda x: f"{x*100:.1f}%")
comparison['Accuracy'] = comparison['Accuracy'].round(3)

print("\nPerformance Comparison:")
print("-" * 60)
display(comparison[['Model', 'Accuracy', 'Accuracy (%)', 'Interpretability', 'Complexity']])


# Visual Comparison
# ============================================

print("\n Visual Comparison")
print("-" * 40)

import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))


models = ['Logistic\nRegression', 'Decision\nTree', 'Random\nForest']
accuracies = [accuracy_logreg, accuracy_tree, accuracy_forest]

bars = ax1.bar(models, accuracies, color=['lightblue', 'lightgreen', 'salmon'], edgecolor='black')
ax1.set_title('Model Accuracy Comparison', fontweight='bold', fontsize=14)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_ylim([0.7, 0.9])
ax1.grid(axis='y', alpha=0.3)


for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.005,
            f'{acc:.3f}\n({acc*100:.1f}%)',
            ha='center', va='bottom', fontweight='bold')


best_idx = accuracies.index(max(accuracies))
best_model_name = models[best_idx].replace('\n', ' ')
best_predictions = [y_pred_logreg, y_pred_tree, y_pred_forest][best_idx]

cm = confusion_matrix(y_test, best_predictions)


labels = ['Did Not\nSurvive', 'Survived']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels,
            cbar_kws={'label': 'Count'}, ax=ax2)

ax2.set_title(f'Confusion Matrix - {best_model_name}\n(Best Model)',
              fontweight='bold', fontsize=14)
ax2.set_xlabel('Predicted Label', fontsize=12)
ax2.set_ylabel('True Label', fontsize=12)

plt.tight_layout()
plt.show()


# Best Model Analysis
# ============================================

print("\n" + "="*50)
print("Best Model")
print("="*50)

best_accuracy = max(accuracies)
best_model_idx = accuracies.index(best_accuracy)
best_model_names = ['Logistic Regression', 'Decision Tree', 'Random Forest']
selected_best_model = best_model_names[best_model_idx]

print(f"\n BEST PERFORMING MODEL: {selected_best_model}")
print(f"   • Accuracy: {best_accuracy:.3f} ({best_accuracy*100:.1f}%)")
print(f"   • Improvement over baseline: {(best_accuracy - accuracy_logreg)*100:.1f}% points")

if selected_best_model == 'Random Forest':
    print("\nDetailed Classification Report for Random Forest:")
    print("-" * 60)
    print(classification_report(y_test, y_pred_forest,
                               target_names=['Did Not Survive', 'Survived']))


    print("\n Random Forest provides feature importance scores")
    print("   (Will analyze in detail in the next section)")

elif selected_best_model == 'Decision Tree':
    print("\n Decision Tree Rules Analysis:")
    print("   • Can visualize decision paths")
    print("   • Shows clear if-then rules")

else:
    print("\n Logistic Regression Coefficients:")
    print("   • Shows feature impact on survival probability")
    print("   • Easy to interpret and explain")


# Learning Insights
# ============================================

print("\n" + "="*50)
print(" INSIGHTS")
print("="*50)

print("\n1. Model Performance Progression:")
print(f"   • Baseline (Logistic Regression): {accuracy_logreg*100:.1f}%")
print(f"   • Simple Tree (Decision Tree):    {accuracy_tree*100:.1f}%")
print(f"   • Ensemble (Random Forest):       {accuracy_forest*100:.1f}%")

print("\n2. Observations:")
if accuracy_forest > accuracy_tree > accuracy_logreg:
    print("   - Ensemble methods improve upon single models")
    print("   - More complex models capture patterns better")
elif accuracy_tree > accuracy_forest:
    print("   - Sometimes simpler models perform well")
    print("   - May indicate overfitting in complex models")
else:
    print("   - All models show reasonable performance")
    print("   - Titanic survival has clear predictive patterns")

print("\n3. Business Impact:")
print(f"   • Best model predicts survival with {best_accuracy*100:.1f}% accuracy")
print(f"   • Could correctly identify {int(best_accuracy * len(y_test))} of {len(y_test)} test passengers")
print("   • Provides actionable insights for safety planning")



<a id="model-evaluation"></a>
## 9. Model Evaluation & Performance Analysis

In this section, we perform comprehensive evaluation of our trained models to:
1. **Validate model robustness** through cross-validation
2. **Analyze feature importance** to understand prediction drivers
3. **Examine errors** to identify model weaknesses
4. **Calculate advanced metrics** (ROC-AUC) for thorough assessment

This evaluation ensures our model is reliable and provides insights into its decision-making process.

In [ ]:
# ============================================
# 9. MODEL EVALUATION & PERFORMANCE ANALYSIS
# ============================================

print(" Model Evaluation ")
print("=" * 50)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


if 'accuracy_forest' in locals() and accuracy_forest >= accuracy_tree and accuracy_forest >= accuracy_logreg:
    best_model = forest
    best_model_name = "Random Forest"
    y_pred_best = y_pred_forest
elif 'accuracy_tree' in locals() and accuracy_tree >= accuracy_logreg:
    best_model = tree
    best_model_name = "Decision Tree"
    y_pred_best = y_pred_tree
else:
    best_model = logreg
    best_model_name = "Logistic Regression"
    y_pred_best = y_pred_logreg

print(f"Evaluating best model: {best_model_name}")
print("-" * 40)

# ============================================
# 1. Cross-Validation for Robustness
# ============================================

print("\n1 Cross-Validation")
print("-" * 30)

cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"5-Fold Cross-Validation Scores:")
for i, score in enumerate(cv_scores, 1):
    print(f"   Fold {i}: {score:.3f}")

print(f"\n   Mean CV Accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
print(f"   Test Accuracy:    {accuracy_score(y_test, y_pred_best):.3f}")

# ============================================
# 2. Feature Importance Analysis
# ============================================

print("\n2 Prediction Drivers")
print("-" * 30)

if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    features = X_train_scaled.columns


    top_features = pd.DataFrame({
        'Feature': features,
        'Importance': importances
    }).sort_values('Importance', ascending=False).head(5)

    print("Top 5 Most Important Features:")
    for idx, row in top_features.iterrows():
        print(f"   • {row['Feature']}: {row['Importance']:.3f}")


    plt.figure(figsize=(8, 4))
    plt.barh(top_features['Feature'][::-1], top_features['Importance'][::-1])
    plt.xlabel('Importance Score')
    plt.title(f'Top 5 Features - {best_model_name}')
    plt.tight_layout()
    plt.show()

elif hasattr(best_model, 'coef_'):
    print("Model coefficients available (logistic regression)")
    print("Top features by coefficient magnitude...")

# ============================================
# 3. Error Analysis
# ============================================

print("\n3️Model Weaknesses")
print("-" * 30)

cm = confusion_matrix(y_test, y_pred_best)
tn, fp, fn, tp = cm.ravel()

print(f"Confusion Matrix:")
print(f"               Predicted")
print(f"              No     Yes")
print(f"Actual No   [{tn:3d}]   [{fp:3d}]")
print(f"       Yes  [{fn:3d}]   [{tp:3d}]")

print(f"\nError Analysis:")
print(f"   • Total Errors: {fp + fn}")
print(f"   • False Positives: {fp} (predicted survive, actually died)")
print(f"   • False Negatives: {fn} (predicted die, actually survived)")

# ============================================
# 4. ROC-AUC Analysis
# ============================================

print("\nROC-AUC ANALYSIS (Advanced Metrics)")
print("-" * 30)

if hasattr(best_model, 'predict_proba'):
    y_prob = best_model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    roc_auc = auc(fpr, tpr)

    print(f"ROC-AUC Score: {roc_auc:.3f}")

    # Plot ROC curve
    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Chance')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {best_model_name}')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

    # Interpret AUC
    if roc_auc >= 0.8:
        print("   Interpretation: Good discriminative power")
    elif roc_auc >= 0.7:
        print("   Interpretation: Fair discriminative power")
    else:
        print("   Interpretation: Limited discriminative power")
else:
    print("ROC-AUC not available (model doesn't provide probabilities)")

# ============================================
# EVALUATION SUMMARY
# ============================================


final_accuracy = accuracy_score(y_test, y_pred_best)
print(f"\nBest Model: {best_model_name}")
print(f"Test Accuracy: {final_accuracy:.3f} ({final_accuracy*100:.1f}%)")
print(f"Cross-Validation Consistency: {cv_scores.std():.3f} std dev")

if 'roc_auc' in locals():
    print(f"ROC-AUC Score: {roc_auc:.3f}")

print(f"\nModel achieves {final_accuracy*100:.1f}% accuracy")
print("with reasonable generalization (CV scores consistent).")

<a id="conclusions--recommendations"></a>
## 10. Conclusions & Recommendations

This final section summarizes our predictive analysis of Titanic survival data and provides actionable insights.

In [ ]:
# ============================================
# 10. CONCLUSIONS & RECOMMENDATIONS
# ============================================

print(" Conclusions & Recommendations")
print("=" * 50)


final_accuracy = accuracy_score(y_test, y_pred_best)

print("\n Project Summary")
print("-" * 30)
print(f"• Analysis: Titanic survival prediction")
print(f"• Dataset: {len(df)} passengers, {df.shape[1]} features")
print(f"• Best Model: {best_model_name}")
print(f"• Final Accuracy: {final_accuracy*100:.1f}%")

print("\n Key Findings")
print("-" * 30)
print("1. **Demographics Matter Most:**")
print("   • Gender: Women 74.2% vs Men 18.9% survival")
print("   • Age: Children had 59.0% survival rate")
print("   • Class: 1st class 62.6% vs 3rd class 24.2%")

print("\n2. **Social Factors Influence Survival:**")
print("   • Traveling alone reduced chances")
print("   • Family size affected group dynamics")
print("   • Socioeconomic status was significant")

print("\n3. **Model Performance:**")
print(f"   • Achieved {final_accuracy*100:.1f}% prediction accuracy")
print("   • Most errors: borderline cases (elderly, solo travelers)")
print("   • Strongest predictor: Passenger gender")

print("\n Business Insight")
print("-" * 30)
print("1. **Historical Validation:**")
print("   • 'Women and children first' was followed")
print("   • Class disparities evident in survival rates")

print("\n2. **Predictive Value:**")
print("   • Machine learning can analyze historical patterns")
print("   • Data-driven insights complement historical records")

print("\n3. **Modern Applications:**")
print("   • Similar analysis for disaster preparedness")
print("   • Safety planning considering vulnerable groups")

print("\n LIMITATIONS & FUTURE WORK")
print("-" * 30)
print("• **Data:** Historical (1912), some missing values")
print("• **Scope:** Single event, limited sample size")
print("• **Future:** Apply to other disasters, add more features")

